In [12]:
pip install mediapipe

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/ee/ff/6f914bd788d2be5f22c47952804b0c8d68ef438a7c7fb80df6773866fa2e/mediapipe-0.10.3-cp311-cp311-macosx_11_0_universal2.whl.metadata
  Obtaining dependency information for opencv-contrib-python from https://files.pythonhosted.org/packages/9c/85/fcae5a3cf1ef35f6e4b7d9062d57b1f8312d9f35274f333f1aceb2b0bc50/opencv_contrib_python-4.8.0.74-cp37-abi3-macosx_11_0_arm64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 MB 6.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 5.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.3
    Uninstalling protobuf-4.23.3:
      Successfully uninstalled protobuf-4.23.3
Note: you may need 

In [20]:

import argparse
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mediapipe as mp
# Used to convert protobuf message to a dictionary.
from google.protobuf.json_format import MessageToDict


In [14]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [43]:
cap = cv2.VideoCapture(1)
  
while True:
    # Read video frame by frame
    success, img = cap.read()
  
    # Flip the image(frame)
    img = cv2.flip(img, 1)
  
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
    results = hands.process(imgRGB)

    if results.multi_hand_landmarks:# If hands are present in image(frame)

        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):# Loop through each detected hand
            label = MessageToDict(results.multi_handedness[i])['classification'][0]['label']# Return whether it is Right or Left Hand

            x_min, y_min, x_max, y_max = float('inf'), float('inf'), float('-inf'), float('-inf')# Draw bounding box around the hand
            for landmark in hand_landmarks.landmark:
                x, y = int(landmark.x * img.shape[1]), int(landmark.y * img.shape[0])
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            
            label_size = cv2.getTextSize(label + ' Hand', cv2.FONT_HERSHEY_COMPLEX, 0.9, 2)[0]
            
            if len(results.multi_handedness) == 2:
                cv2.putText(img, 'Both Hands', (450, 250),# Display 'Both Hands' on the image
                        cv2.FONT_HERSHEY_COMPLEX,
                        0.9, (0, 255, 0), 2)
            else:
                if label == 'Left':
                    cv2.putText(img, label + ' Hand',
                            (x_min, y_min - label_size[1]),
                            cv2.FONT_HERSHEY_COMPLEX, 
                            0.9, (0, 255, 0), 2)
                if label == 'Right':
                    cv2.putText(img, label + ' Hand', (x_min, y_min - label_size[1]),
                            cv2.FONT_HERSHEY_COMPLEX,
                            0.9, (0, 255, 0), 2)
  

    cv2.imshow('Video', img)# Display the image(frame)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()